In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist

# Tải dữ liệu MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Chuẩn hóa dữ liệu

# Xây dựng mô hình MLP
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Chuyển ma trận 28x28 thành vector 784
    Dense(128, activation="relu"),  # Lớp ẩn với 128 neuron và hàm ReLU
    Dense(10, activation="softmax") # Lớp đầu ra với 10 neuron cho 10 chữ số
])

# Compile mô hình
model.compile(optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

# Huấn luyện mô hình
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Đánh giá mô hình
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Độ chính xác trên tập kiểm tra: {test_acc:.2%}")


ModuleNotFoundError: No module named 'tensorflow'